In [1]:
type Node{T}
    left::Nullable{Node{T}}
    right::Nullable{Node{T}}
    value::T
end

Node{T}(v::T) = Node(Nullable{Node{T}}(), Nullable{Node{T}}(), v)

Node{T}

In [1]:
n1 = Node(Nullable(Node(2)), Nullable(Node(3)), 10)

LoadError: LoadError: UndefVarError: Node not defined
while loading In[1], in expression starting on line 1

In [12]:
left(n::Node) = isnull(n.left) ? Void(): get(n.left)
right(n::Node) = isnull(n.right) ? Void(): get(n.right)

right (generic function with 1 method)

In [91]:
function search{T}(graph::Node{T}, number)
    for i = 1:10
        if graph.value == number
            return number
        else
            try
                graph = graph.left.value
            catch
                return 0
            end
        end
    end
end
    

search (generic function with 1 method)

In [1]:
include("SymmetricMatrix.jl")
include("Test.jl")
using Test

In [1]:
include("SymmetricMatrix.jl")
using SymmetricMatrix
using NullableArrays
importall SymmetricMatrix

In [3]:
using Iterators
function genstensor(T::Type, dims::Int, l::Int, seed::Int = 1234)
    srand(seed)
    ret = zeros(T, fill(l, dims)...)
    indices = collect(product(fill(collect(1:l), dims)...))
    elements = Array{T,1}(randn(l^dims÷dims))
    r = 1
    for i in indices 
        if issorted(i)          
            for k in collect(permutations(i))
                ret[k...] = elements[r]
            end
            r += 1
        end
    end
    ret
end

genstensor (generic function with 2 methods)

In [4]:
st = genstensor(Float64, 4,100);
st1 = genstensor(Float64, 4,100, 1233);

In [5]:
bs = (convert(BoxStructure{Float64}, st, 10)) ;
@time bs1 = (convert(BoxStructure{Float64}, st1, 10)) ;

  4.022548 seconds (335.30 k allocations: 9.016 GB, 4.74% gc time)


In [10]:
@time bs+bs1;
@time bs+2;
@time st + st1;
@time st + 2;

  0.176726 seconds (326.84 k allocations: 77.425 MB, 31.42% gc time)
  0.145968 seconds (306.82 k allocations: 76.618 MB, 33.04% gc time)
  0.226577 seconds (7 allocations: 762.940 MB, 16.91% gc time)
  0.175412 seconds (7 allocations: 762.940 MB, 24.00% gc time)


In [11]:
@time convert(Array{Float64}, bs);
@time vec(bs);
@time vec(st);

  0.878337 seconds (1.22 M allocations: 1.541 GB, 13.66% gc time)
  0.849561 seconds (1.22 M allocations: 1.541 GB, 9.76% gc time)
  0.000001 seconds (6 allocations: 240 bytes)


In [13]:
@time bs.*bs1;
@time bs+bs1;
@time bs./bs1;
@time st.*st1;
@time st+st1;
@time st./st1;

  0.091177 seconds (331.13 k allocations: 77.687 MB)
  0.139853 seconds (326.84 k allocations: 77.425 MB, 39.03% gc time)
  0.170148 seconds (331.13 k allocations: 77.687 MB, 23.96% gc time)
  0.298498 seconds (13 allocations: 762.940 MB, 13.26% gc time)
  0.217914 seconds (7 allocations: 762.940 MB, 20.48% gc time)
  0.814043 seconds (13 allocations: 762.940 MB, 4.22% gc time)


In [2]:
t = genstensor(Float64, 4,10)

LoadError: LoadError: UndefVarError: genstensor not defined
while loading In[2], in expression starting on line 1

In [2]:
a = randn(10,10)
b = convert(BoxStructure{Float64}, a*a');
c = randn(10,8);

In [3]:
tt(b,c) - a*a'*c

10x8 Array{Float64,2}:
 -1.77636e-15   0.0           0.0          …  -4.44089e-16  -8.88178e-16
 -1.11022e-15   0.0           0.0             -8.88178e-16  -1.77636e-15
 -1.77636e-15   0.0           0.0             -1.55431e-15   0.0        
  0.0           7.10543e-15   0.0              3.55271e-15   8.88178e-16
 -1.77636e-15  -5.32907e-15   0.0             -8.88178e-16   0.0        
  0.0           0.0           0.0          …   0.0           1.77636e-15
 -3.55271e-15   0.0           0.0             -1.77636e-15   0.0        
 -1.77636e-15   0.0           0.0              2.66454e-15  -1.77636e-15
  1.77636e-15  -4.44089e-15  -1.77636e-15      8.88178e-16   0.0        
 -8.88178e-16   4.44089e-16   0.0              1.11022e-16   0.0        